In [1]:
import numpy as np, pandas as pd, sqlite3
from sklearn.mixture import GaussianMixture as GMM
from sklearn.datasets._samples_generator import make_blobs
from sklearn import metrics

In [139]:
df = pd.read_csv('data.csv', index_col=0)
df.head()

,symbol,1,2,3,4,5,6,7,8,9,...,324,325,326,327,328,329,330,331,332,333
0,575,-0.001137,0.008527,-0.005808,-0.003630,0.010863,0.005933,-0.002473,0.002579,-0.001755,...,-0.012214,-0.003507,0.003961,0.001404,-0.006647,0.001274,0.009193,-0.002167,-0.006118,0.001938
1,537,0.010103,-0.001521,-0.001524,0.003354,0.005616,0.001217,-0.001517,0.002732,-0.000304,...,-0.019349,-0.005060,-0.004344,0.002893,-0.001817,-0.001446,0.001454,-0.003252,0.010388,0.011544
2,649,0.010156,0.003907,-0.006261,-0.002778,0.005568,0.001522,-0.002581,0.001057,0.000497,...,-0.000754,0.001300,0.003704,0.007972,0.000918,-0.009861,-0.009649,-0.009703,0.000843,0.015026
3,1,0.009744,0.002821,0.005121,0.005097,0.005989,0.002376,-0.000714,0.001137,-0.003807,...,-0.004954,-0.003002,0.000000,0.003549,0.001353,0.002059,0.003372,-0.003671,-0.005664,0.000903
4,132,0.005622,0.000148,0.001327,0.002505,0.001913,0.000883,0.001291,0.002204,-0.001176,...,-0.000938,0.000469,0.001251,-0.002692,-0.002502,0.002817,0.001564,0.000000,0.000000,0.000000


In [142]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
arr_scaled = scale.fit_transform(df.iloc[:,1:])
df_scaled = pd.DataFrame(arr_scaled)
df_scaled.insert(loc=0, column='symbol', value=df['symbol']) # Insert ticker column
df_scaled.head()

,symbol,0,1,2,3,4,5,6,7,8,...,323,324,325,326,327,328,329,330,331,332
0,575,-0.648018,0.950573,-1.278249,-0.424756,1.308712,1.029614,-0.784523,0.093321,-0.167799,...,-0.471970,0.235687,0.941589,-0.182861,-1.645517,0.180078,1.471469,0.088403,-0.909620,-0.210074
1,537,0.531791,-0.798194,-0.640297,0.310759,0.550993,0.012219,-0.576396,0.124944,0.120949,...,-1.251706,0.020983,-0.641875,0.123572,-0.620200,-0.499707,-0.017904,-0.131772,2.665136,1.640602
2,649,0.537435,0.146508,-1.345765,-0.335097,0.544121,0.077932,-0.808148,-0.222339,0.280198,...,0.780444,0.900275,0.892432,1.168409,-0.039459,-2.602787,-2.154642,-1.440953,0.597831,2.311504
3,1,0.494101,-0.042419,0.349378,0.494310,0.604832,0.262265,-0.401481,-0.205700,-0.575930,...,0.321438,0.305458,0.186302,0.258558,0.052871,0.376225,0.351244,-0.216853,-0.811402,-0.409514
4,132,0.061490,-0.507811,-0.215704,0.221325,0.016416,-0.059846,0.035204,0.015456,-0.052533,...,0.760336,0.785310,0.424777,-1.025505,-0.765566,0.565702,0.003303,0.528106,0.415330,-0.583472
